In [39]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import string
import random
from random import randint
# import torch.backends.cudnn as cudnn

In [40]:
# Read data into string and seperate train and valid data
with open('text8') as file:
    data = file.read()
valid_size = 1000
valid_text = data[:valid_size]
train_text = data[valid_size:]
train_size = len(train_text)
valid_size = len(valid_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


In [41]:
# Utility functions
vocab_size = len(string.ascii_lowercase) + 1 # 0 index for ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter + 1
    elif char == ' ':
        return 0
    else:
        print('Unexpected character %s' % char)
        return 0

def id2char(dictid):
    if dictid > 0:
        return chr(dictid + first_letter - 1)
    else: return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character ï
1 26 0 0
a z  


In [42]:
batch_size = 64
num_unrollings = 10

# Generate batches parallely across the text at equal intervals
# Each batch contains one character from each of the positions
# Positions are updated after generating every batch
# The next batch would therefore contain the next characters from all the chosen positions
# num_unrollings number of batches are processed at once
# Each character is represented as a one hot vector
class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [offset*segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
        
    def _next_batch(self):
        batch = np.zeros(shape=(self._batch_size, vocab_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b]])] = 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
    
    def _next(self):
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

In [43]:
dtype = torch.FloatTensor
hidden_size = 512 # 1024 for gpu 

class LSTM(nn.Module):
    
    def __init__(self, vocab_size, hidden_size):
        super(LSTM, self).__init__()
        # LSTM architecture
        self.inputW = nn.Linear(vocab_size, hidden_size)
        self.inputU = nn.Linear(hidden_size, hidden_size)
        
        self.forgetW = nn.Linear(vocab_size, hidden_size)
        self.forgetU = nn.Linear(hidden_size, hidden_size)
        
        self.outputW = nn.Linear(vocab_size, hidden_size)
        self.outputU = nn.Linear(hidden_size, hidden_size)
        
        self.cellW = nn.Linear(vocab_size, hidden_size)
        self.cellU = nn.Linear(hidden_size, hidden_size)
        
        self.softW = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(0)
        
    def forward(self, one_hot_input, cell_prev, hidden_prev):
        sig = nn.Sigmoid()
        tnh = nn.Tanh()
        cell_prev = Variable(cell_prev)
        hidden_prev = Variable(hidden_prev)
        input_gate = sig(self.inputW(one_hot_input) + self.inputU(hidden_prev))
        forget_gate = sig(self.forgetW(one_hot_input) + self.forgetU(hidden_prev))
        output_gate = sig(self.outputW(one_hot_input) + self.outputU(hidden_prev))
        update = tnh(self.cellW(one_hot_input) + self.cellU(hidden_prev))
        cell = (forget_gate * cell_prev) + (input_gate * update)
        hidden = output_gate * tnh(cell)
        logits = self.softW(hidden)
        logits = self.dropout(logits)
        return cell.data, hidden.data, logits

lstm = LSTM(vocab_size, hidden_size)
# lstm.cuda()

In [44]:
class StateManager(object):
    def __init__(self):
        self.hidden_state = torch.zeros(batch_size, hidden_size).type(dtype)
        self.cell_state = torch.zeros(batch_size, hidden_size).type(dtype)
        
    def save_state(self, cell_state, hidden_state):
        self.cell_state = cell_state
        self.hidden_state = hidden_state
        
    def load_state(self):
        return self.cell_state, self.hidden_state
    
    def reset_state(self):
        self.hidden_state = torch.zeros(batch_size, hidden_size).type(dtype)
        self.cell_state = torch.zeros(batch_size, hidden_size).type(dtype)
        
sm = StateManager()

In [45]:
learning_rate = 0.1
train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

def train():
    cell, hidden = sm.load_state()
    batches = train_batches._next()
    optimizer.zero_grad()
    lstm.zero_grad()
    loss_function = nn.CrossEntropyLoss()
    loss = 0
    for u in range(num_unrollings):
        one_hot_input = Variable(torch.from_numpy(batches[u]).type(dtype), requires_grad=False)
        cell, hidden, logits = lstm(one_hot_input, cell, hidden)
        # cell, hidden, logits = lstm(one_hot_input.cuda(), cell.cuda(), hidden.cuda())
        labels = Variable(torch.from_numpy(np.argmax(batches[u+1], axis=1)))
        loss += loss_function(logits, labels)
        # loss += loss_function(logits.cuda(), labels.cuda())
    loss.backward()
    torch.nn.utils.clip_grad_norm(lstm.parameters(), 5)
    optimizer.step()
    sm.save_state(cell, hidden)
    return loss / num_unrollings

In [46]:
# Generate a random character and feed to model
# Take predicted character and feed it back again to generate subsequent characters

def sample_distribution(distribution):
# Sample one element from a distribution assumed to be an array of normalized probabilities
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def sample():
    charid = randint(0, vocab_size-1)
    print(id2char(charid), end='')
    cell = torch.zeros(1, hidden_size).type(dtype)
    hidden = torch.zeros(1, hidden_size).type(dtype) 
    soft = nn.Softmax(dim=1)
    for i in range(100):
        one_hot = torch.zeros(1, vocab_size).type(dtype)
        one_hot[0, charid] = 1.0
        one_hot = Variable(one_hot, requires_grad=False)
        cell, hidden, logits = lstm(one_hot, cell, hidden)
        # cell, hidden, logits = lstm(one_hot.cuda(), cell.cuda(), hidden.cuda())
        output = soft(logits)
        charid = sample_distribution(output.data[0])
        print(id2char(charid), end='')
    print(' ')

In [47]:
valid_batches = BatchGenerator(valid_text, 1, 1)

def valid_perplexity():
    loss_function = nn.CrossEntropyLoss()
    loss = 0
    cell = torch.zeros(1, hidden_size).type(dtype)
    hidden = torch.zeros(1, hidden_size).type(dtype)
    for i in range(valid_size):
        batches = valid_batches._next()
        one_hot_input = Variable(torch.from_numpy(batches[0]).type(dtype), requires_grad=False)
        cell, hidden, logits = lstm(one_hot_input, cell, hidden)
        # cell, hidden, logits = lstm(one_hot_input.cuda(), cell.cuda(), hidden.cuda())
        labels = Variable(torch.from_numpy(np.argmax(batches[1], axis=1)))
        loss += loss_function(logits, labels)
        # loss += loss_function(logits.cuda(), labels.cuda())
    return torch.exp(loss / valid_size)

In [48]:
num_iters = 50001 #50001, no dropout, lr=0.1

cudnn.benchmark = True
cudnn.fasttest = True

for i in range(num_iters):
    lstm.train()
    l = train()
    if i%1000 == 0: 
        print('Average loss at step %d: %.3f ' % (i,l))
        print('Minibatch perplexity: %.3f' % torch.exp(l))
        print('Validation perplexity: %.3f' % valid_perplexity())
        lstm.eval()
        sample()
        print('')

Average loss at step 0: 3.291 
Minibatch perplexity: 26.857
Validation perplexity: 24.429
lzqvogzfzrsgeiiahhrpkg vvekasvgmargsdpmid jjoegytikqebmr y y pdvzvmajzskgruoeykcfopxxiorklujxsgtsijpe 



KeyboardInterrupt: 